In [1]:
try: 
  from tabulate import tabulate
except ImportError:
  !pip install tabulate
  from tabulate import tabulate

In [2]:
class MSANonOptimal:
  
  def __init__(self, aa):
    self.aa = aa


  def cmp(self, a, b):
      """Returns the score for comparing a and b, which may be characters or the gap symbol.
      cmp defines the scoring scheme for the string distance."""
      if a=="-" or b=="-":
          return 1
      else:
          return int(a!=b)
        
        
  def sum_of_pairs(self, aa,bb):
    """Computes the sum of pairs of all characters in string aa against all characters in string bb.
    aa and bb are alignments and may contain the gap symbol."""
    c=0
    for a in aa:
        for b in bb:
            c+=self.cmp(a, b)
    return c
  
  def min_dir(self, a,b,c):
    """Returns the minimum value of a, b, and c and which are minimal.

    c is encoded as NW, b as N and a as W.
    More than one can be the minimum.
    Preference is given to NW, then N, then W.
    This means that alignments with match/mismatch
    from end to front are preferred. If all alignments
    are needed, then the function has to return a set of directions
    instead of single direction."""
    if c==min(a,b,c):
        return (c,"NW")
    if b==min(a,b,c):
        return (b,"W")
    if a==min(a,b,c):
        return (a,"N")
      
  def get(self, a, i):
    """Returns the list of characters at position i in the alignment a, which is a list of strings."""
    return list(map(lambda s: s[i], a))
  
  def add(self, a, aas):
    """Given a list of n strings (an alignment) and a list of n characters, add appends character i to string i.""" 
    a2=[]
    for i in range(len(a)):
        a2.append(a[i]+aas[i])
    return a2
  
  def align(self, a,b,d_dir,i,j):
    """Outputs the alignment of alignments a and b up to position i and j given the direction matrix d_dir.
    Alignments a and b are lists of strings with the original character sequences possibly with gaps."""
    if i==0 and j==0:
        return [[""]*len(a),[""]*len(b)]
    elif d_dir[i,j]=="W":
        (a2,b2)=self.align(a,b,d_dir,i,j-1)
        return [self.add(a2, self.gap(a)), self.add(b2, self.get(b,j-1))]
    elif d_dir[i,j]=="N":
        (a2,b2)=self.align(a,b,d_dir,i-1,j)
        return [self.add(a2,self.get(a,i-1)), self.add(b2,self.gap(b))]
    elif d_dir[i,j]=="NW":
        (a2,b2)=self.align(a,b,d_dir,i-1,j-1)
        return [self.add(a2,self.get(a,i-1)), self.add(b2,self.get(b,j-1))]
      
  def gap(self, a):
    """Returns n gaps for an alignment of n sequences."""
    return ["-"]*len(a)
  
  def print_matrix(self, a,b,d,m,n):
    matrix=[]
    for s in b:
        matrix.append([""]*len(a) + [""] + list(s))
    for i in range(m+1):
        row=[]
        if i==0:
            row+=[""]*len(a)
        else:
            row+=self.get(a,i-1)
        for j in range(n+1):
            row.append(d[(i,j)])
        matrix.append(row)
    print()
    print("Alignment of "+", ".join(a)+" with "+", ".join(b))
    print()
    print(tabulate(matrix))
 
  def lev2(self, a, b):
    """Align the two alignments a and b and return the common alignment and its score."""
    m = len(a[0])
    n = len(b[0])
    d = dict()
    d_dir = dict()
    d[(0, 0)] = 0
    d_dir[(0, 0)] = ""
    for i in range(1, m + 1):
        d[(i, 0)] = i*len(a)*len(b)+ int(i*len(b)*(len(b)-1)/2)
        d_dir[(i, 0)] = "N"
    for j in range(1, n + 1):
        d[(0, j)] = j*len(a)*len(b) + int(j*len(a)*(len(a)-1)/2)
        d_dir[(0, j)] = "W"
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            (d[(i, j)],d_dir[(i, j)]) = self.min_dir(d[(i - 1, j)] + self.sum_of_pairs(self.get(a,i-1),self.gap(b)) + int(len(b)*(len(b)-1)/2),
                                            d[(i, j - 1)] + self.sum_of_pairs(self.gap(a),self.get(b,j-1)) + int(len(a)*(len(a)-1)/2),
                                            d[(i - 1, j - 1)] + self.sum_of_pairs(self.get(a,i-1),self.get(b,j-1)))
    self.print_matrix(a,b,d,m,n)
    (a1,a2)=self.align(a,b,d_dir,m,n)
    a1.extend(a2)
    return (d[(m, n)], a1)
  
 
  def msa(self):
    """Progressive alignment of the sequences in a.

    Succesively, the sequences in a are added to an alignment.
    The final alignment and the sum of the scores are returned."""
    b=[self.aa[0]]
    total = 0
    for i in range(1,len(self.aa)):
        (score,b) = self.lev2(b,[self.aa[i]])
        total += score
    return (total,b)

In [3]:
from typing import List


class Node:
    """
    Node class for the A*-Search. 
    Consists of a column of characters (cc). Each node is also characterized by the
    present cost, by the underestimated cost to the aim and by the total cost g + h.
    Furthermore we have "pointer" k1, k2... , which points to the character in the total strings to be aligned,
    which needs to be added next. If the strings to be aligned have lengths i1, i2 ... the final states are
    characterized by the condition i1 == k1.
    The method __lt__ needs to be defined in order to implement an order on the nodes, which is necessary for dealing
    with the PriorityQueue.
    """

    def __init__(self, cc: List[str], kk: List[int], g: int, h: float, pred: 'Node'):
        self.cc = cc
        self.kk = kk
        self.h = h
        self.g = g
        self.f = g + h
        self.pred = pred

    def __lt__(self, other: 'Node'):
        return self.f < other.f

    def is_endnode(self, aa: List[str]):
        return self.kk == [len(aa[i]) for i in range(len(aa))]

In [4]:
import queue as Q
import itertools

class MSAOptimal:
  """
  Implements the optimal multiple sequence alignment algorithm.
  The class is initialized by passing the strings to be aligned to the constructor.
  """

  def __init__(self, aa: List[str]):
    self.aa = aa
    start_node = Node(cc=[""]*len(aa), 
                      kk=[0]*len(aa), 
                      g=0, 
                      h=self.underestimation(aa), 
                      pred=None)
    self.start_node = start_node
    # create a priority queue, the elements are ordered by the total cost f
    queue = Q.PriorityQueue()
    queue.put(start_node)
    self.queue = queue

  def append_neighbors(self, node: Node):
    # get next characters to be added, which are aa[i][node.kk[i]]
    # if in any string there are no characters left use the gap symbol "-"
    next_characters = [self.aa[i][node.kk[i]] if node.kk[i] < len(self.aa[i]) else "-"
                       for i in range(len(self.aa))]
    # get all combinations of possibilities to choose from next_characters
    combinations = [y for y in itertools.product([True, False], 
                                                 repeat=len(self.aa))]
    # remove the characters to be added, which only consist of gaps
    combinations.remove((False, )*len(self.aa))

    for comb in combinations:
        # get one new neighbor for each combination of True/False
        # if the corresponding index in comb is True, take the this character into the next node
        nodecc = [next_characters[i] if comb[i] else "-" 
                  for i in range(len(next_characters))]
        # eliminate all only-gap character sequences
        if nodecc != ["-"]*len(self.aa):
          # if comb[i] is true, we need to add 1 to the next nodes indices at the corresponding position
          # this is done only, if the i-th string has remaining characters
          nodekk = [node.kk[i] + int(comb[i]) if node.kk[i]!=len(self.aa[i]) else node.kk[i]
                    for i in range(len(next_characters))
                    ]
          # calculate the new present cost for the node
          gw = node.g + self.transitioncost(nodecc)
          # get remaining strings, which has not been aligned so far
          raaw = [self.aa[j][nodekk[j]:] for j in range(len(self.aa))]
          # calculate underestimation
          hw = self.underestimation(raaw)
          # put the neighbor into the PriorityQueue
          self.queue.put(Node(cc=nodecc, kk=nodekk, g=gw, h=hw, pred=node))

  def cmp(self, a, b):
    if a=="-" or b=="-":
        return 1
    else:
        return int(a != b)

  def transitioncost(self, cc):
    # cc are the characters for the successor
    cost = 0
    for i in range(len(cc)):
      for j in range(i+1, len(cc)):
        cost += self.cmp(cc[i], cc[j])
    return cost

  def underestimation(self, raa):
    """
    raa are the rests of the strings to be aligned.
    """
    m = len(raa)
    sum = 0
    for i in range(m - 1):
        for j in range(i + 1, m):
            sum += self.levdistance(aa=raa[i], bb=raa[j])

    return sum

  def levdistance(self, aa, bb):
    """
    Simplified LevDistance-Algorithm. This class only needs the score not the
    alignment of the rest of the strings
    """
    d = {}
    m = len(aa)
    n = len(bb)

    for i in range(m + 1):
        d[(i, 0)] = i
    for j in range(n + 1):
        d[(0, j)] = j

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            d[(i, j)] = min([d[(i - 1, j)] + 1,
                             d[(i, j - 1)] + 1,
                             d[(i - 1, j - 1)] + int(aa[i - 1] != bb[j - 1])])

    return d[(m, n)]

  def get_path(self, node: Node):
    """
    Given the endnode 'node' backtrace the path to the startnode and return it.
    """
    path = []

    # pred of startnode is set to None
    while node is not None:
        path.insert(0, node)
        node = node.pred
    return path

  def print_alignment(self, path: List[Node]):
    print("Best Alignment: ")

    matrix = []
    for i in range(len(self.aa)):
        matrix.append([x.cc[i] for x in path])

    print(tabulate(matrix))

  def msa_alignment(self):
    """
    The algorithm pops nodes out of the PriorityQueue until an end node is reached.
    """
    n: Node = self.queue.get(block=False)
    while not n.is_endnode(self.aa):
        self.append_neighbors(node=n)

        n = self.queue.get(block=False)
    
    path = self.get_path(n)
    self.print_alignment(path)
    
    print("The total score is " + str(n.f))
    return self.get_path(n), n.f

In [5]:
aa = ["Peter", "Petra", "Petta"]

print("A* Search")
msaoptimal = MSAOptimal(aa)

path, score = msaoptimal.msa_alignment()

print("\n")

print("Greedy Search")
msanonoptimal = MSANonOptimal(aa)

score, m = msanonoptimal.msa()

for mm in m:
  print(mm)
print("The total score is", score)


A* Search
Best Alignment: 
  -  -  -  -  -
  P  e  t  e  r
  P  e  t  r  a
  P  e  t  t  a
  -  -  -  -  -
The total score is 5


Greedy Search

Alignment of Peter with Petra

-  -  -  -  -  -  -
      P  e  t  r  a
   0  1  2  3  4  5
P  1  0  1  2  3  4
e  2  1  0  1  2  3
t  3  2  1  0  1  2
e  4  3  2  1  1  2
r  5  4  3  2  1  2
-  -  -  -  -  -  -

Alignment of Peter, Petra with Petta

-  -  --  -  -  -  --  --
          P  e  t  t   a
      0   3  6  9  12  15
P  P  2   0  3  6  9   12
e  e  4   2  0  3  6   9
t  t  6   4  2  0  3   6
e  r  8   6  4  2  2   5
r  a  10  8  6  4  4   3
-  -  --  -  -  -  --  --
Peter
Petra
Petta
The total score is 5
